In [34]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib

matplotlib.rcParams['figure.figsize'] = [16, 12]

import math
import numpy as np

from interaction import Interaction
from environment import Environment
from fish import Fish
from channel import Channel
from observer import Observer
from dynamics import Dynamics

from utils import generate_distortion, generate_fish, run_simulation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
#from events import Homing

run_time = 30 # in seconds
num_fish = 50
arena_size = np.array([1780, 1780, 1170])
arena_center = arena_size / 2.0
initial_spread = 3000
fish_pos = initial_spread * np.random.rand(num_fish, 3) + arena_center - initial_spread / 2.0
#fish_pos = np.zeros((num_fish, 3)) + arena_center
fish_speed = 65 #xx old
fish_vel = np.zeros((num_fish, 3)) #fish_speed / 3 * np.random.rand(num_fish, 3)
fish_phi = math.pi * np.random.rand(num_fish, 1) #xx
fish_vphi = np.zeros((num_fish, 1)) #xx
clock_freqs = 1
verbose = False

distortion = generate_distortion(type='none', magnitude=130, n=math.ceil(arena_size[0]/10)+1, show=False)
environment = Environment(
    arena_size=arena_size,
    node_pos=fish_pos,
    node_vel=fish_vel,
    node_phi=fish_phi, #xx
    node_vphi=fish_vphi,#xx
    distortion=distortion,
    prob_type='binary',
    conn_thres=2500,
    conn_drop=1,
    noise_magnitude=10,
    verbose=verbose
)
interaction = Interaction(environment, verbose=verbose)
channel = Channel(environment)
dynamics = Dynamics(environment, clock_freq=clock_freqs)

fish = generate_fish(
    n=num_fish, 
    channel=channel, 
    interaction=interaction,
    dynamics=dynamics,
    target_dist=130,
    lim_neighbors=[2,3],
    neighbor_weights=1.0,
    fish_max_speeds=fish_speed,
    clock_freqs=clock_freqs,
    verbose=verbose
)
channel.set_nodes(fish)

observer = Observer(fish=fish, environment=environment, channel=channel)

#missing_aircraft = Homing()
#for i in range(1, run_time): 
#    observer.instruct(event=missing_aircraft, rel_clock=i, pos=np.array([1600, 400, 1000]))
    
run_simulation(fish=fish, observer=observer, run_time=run_time, dark=False, white_axis=False, no_legend=True)

Please wait patiently 30 seconds. Thanks.
It's time to say bye bye!


In [36]:
# save data
data = np.zeros((6, run_time, num_fish))

data[0, :, :] = np.transpose(np.array(observer.x))
data[1, :, :] = np.transpose(np.array(observer.y))
data[2, :, :] = np.transpose(np.array(observer.z))
data[3, :, :] = np.transpose(np.array(observer.vx))
data[4, :, :] = np.transpose(np.array(observer.vy))
data[5, :, :] = np.transpose(np.array(observer.vz))

np.save('dynamics', data)